<a href="https://colab.research.google.com/github/echughes529/echughes529/blob/main/Keras_tutorial_Neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural networks**

---


In this tutorial, neural network is used to demonstrate how to use deep learning to predict the energy of a specific configuration of the water molecule. In Keras, there are a number of hyperparameters that can be customize and that affects the performance of your model significantly. We willl explore some of these hyperparameters in this tutorial.

For this tutorial, you are provided with:
1. ``H2O_unrotated.xyz`` and ``H2O_unrotated.ener``: a data set of 1750 configurations of the water molecule with random distortions. All molecules in this set are oriented in the same direction.
2. ``H2O_rotated.xyz`` and ``H2O_rotated.ener``: a data set of 1750 configurations of the water molecule with random distortions, rotations, and translations. Molecules in this set are not oriented in the same direction.

The goal is to have a machine learning model that is capable of predicting the energy of each configurations. This tutorial provides a guide for you to build your own model that can achieve this goal for both data sets.

*(You can run this file to see what the commands do but please **do not** try to modify this file directly here. Your edits will not be saved. Make a copy in your Drive under the File tab in order to edit it.)*

## Building the model

*(If you run this in Google Colab, you need to upload the data files to the content folder and modify the file path accordingly. If you run this on Jupyter Notebook, you need to put the data files and this code file in the same folder.)*

Our model needs to take molecular structure information as input and predict the energy based on this information.

In [1]:
# import necessary libraries
!pip install ase
from ase.io import read
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.2 MB 31.6 MB/s 


In [2]:
file_path="/content/H2O_unrotated.xyz"

ftrs=read(file_path,':')
nsample=len(ftrs)                                                   
natom=[ftrs[i].get_number_of_atoms() for i in range(nsample)]
coords=np.array([ftrs[i].get_positions() for i in range(nsample)])
all_labels=[ftrs[i].get_chemical_symbols() for i in range(nsample)]


# read the corresponding energy values
ener=np.loadtxt("/content/H2O_unrotated.ener")

FileNotFoundError: ignored

For neural network, as the method is numerical, we should normalize our input and output for better accuracy.
$$
x_i'=\frac{x_i - \bar{x}}{x_{max} - x_{min}}
$$

In [ ]:
#normalize inputs
norm_coords=np.zeros((coords.shape))
for a in range(natom[0]):
  for j in range(3):
    norm_coords[:,a,j] = (coords[:,a,j] - np.mean(coords[:,a,j]))/(np.max(coords[:,a,j]) - np.min(coords[:,a,j]))

#normalize outputs
norm_E = (ener - np.mean(ener))/(np.max(ener) - np.min(ener))

First, we build a model with customizable parameters to make our codes short and neat. There are 2 basic ways to build a model in Keras, through the Sequential API and Functional API. Sequential models are straightforward and easy to create by building it layer by layer, but are not flexible (not allowing multiple inputs and outputs, no layer sharings, etc.) On the other hand, the Functional API is more flexible and powerful, allowing different customizations. In this tutorial, for the sake of simplicity, we will use the Sequential API and the `add()` method.

You can test if the model architecture is as desired. In the table of this output, the layers are listed from input, hidden layers, and lastly, output layer. Each layer has its own output shape and the number of parameters that will be learned during the training process. The total number of parameters, trainable parameters, and non-trainable parameters are printed below the table.

In [ ]:
# build and compile the model
def build_compile_model(natom, reg, learn_rate):
    model = keras.Sequential()
    #specify an input layer with the number of nodes = the number of relative distances
    #model.add(layers.Input(shape=(natom,3)))

    #flatten the 2D input array into a 1D array, you can also do this with numpy .flatten()
    model.add(layers.Flatten(input_shape=(natom,3)))

    #add hidden layers
    model.add(layers.Dense(32, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(reg), bias_regularizer=keras.regularizers.l2(reg)))
    model.add(layers.Dense(16, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(reg), bias_regularizer=keras.regularizers.l2(reg)))
    model.add(layers.Dense(8, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(reg), bias_regularizer=keras.regularizers.l2(reg)))

    #finally, add an output layer
    model.add(layers.Dense(1, use_bias=False,kernel_regularizer=keras.regularizers.l2(reg)))

    # define the optimizer: Adam is a gradient descent algorithm 
    opt=keras.optimizers.Adam(learning_rate=learn_rate)

    #compile the model
    model.compile(loss='mse',optimizer=opt)

    return model

model = build_compile_model(3, 1e-6, 0.001)
model.summary()

The model is now ready to be trained. The data set is randomly divided here into the training set (with 1250 configurations), the validation set (with 250 configurations), and the test set (with 250 configurations). The validation set will mainly be used in the later sections to optimize the hyperparameters. The test set contains configurations your model has never seen and is used to test the generalizability of the model and its RMSE.

In [ ]:
idx=np.arange(nsample)
np.random.shuffle(idx)
tr_coords=norm_coords[idx[:1250],:,:]
tr_E=norm_E[idx[:1250]]
va_coords=norm_coords[idx[1250:1500],:,:]
va_E=norm_E[idx[1250:1500]]
te_coords=norm_coords[idx[1500:],:,:]
te_E=ener[idx[1500:]]

history=model.fit(tr_coords,tr_E,epochs=500,validation_data=(va_coords,va_E),verbose=0)

# plot a learning curve of loss of the training and validation set vs the number of epochs the model is trained
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
norm_predict=model.predict(te_coords)
predict=norm_predict*(np.max(ener) - np.min(ener))+np.mean(ener)
rmse = np.sqrt(np.mean((predict-te_E)**2))

print("The model's RMSE is: ", rmse)
#visualize the model's prediction vs the correct value
plt.axes(aspect='equal')
plt.scatter(te_E, predict)
plt.xlabel('Reference Values (Hartree)')
plt.ylabel('Predictions (Hartree)')
plt.plot([np.min([np.min(te_E),np.min(predict)]),np.max([np.max(te_E),np.max(predict)])], [np.min([np.min(te_E),np.min(predict)]),np.max([np.max(te_E),np.max(predict)])], 'r-')
plt.show()

## Effects of hyperparameters

Hyperparameters usually affect the performance of your model significantly. Therefore, it is important to optimize them. Two hyperparameters are optimized in the following sections but (of course) there are many more that can be explored. You are encouraged to play with the values of the hyperparameters.

1. Regularization

In [ ]:
regularization=[1,0.1,0.01,0.001,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9]
va_loss=[]
for r in range(len(regularization)):
  model = build_compile_model(3, regularization[r], 0.001)
  history = model.fit(tr_coords,tr_E,epochs=500,validation_data=(va_coords,va_E),verbose=0)
  va_predict=model.predict(va_coords)
  # get the RMSE of the model prediction for the validation set
  va_loss.append(np.sqrt(np.mean((va_predict-va_E)**2)))

#visualize the effects of different regularization values on the model's accuracy
plt.plot(regularization, va_loss, marker='o')
plt.yscale("log")
plt.xscale("log")
plt.xlabel('Regularization value')
plt.ylabel('Validation loss')
plt.show()

# get the regularization value corresponding to the smallest validation error
for i in range(len(va_loss)):
    if va_loss[i]==min(va_loss):
        reg=regularization[i]


2. Learning rate

In [ ]:
learning_rate=[1,0.1,0.01,0.001,0.0001,1e-5]
va_loss=[]
for lr in range(len(learning_rate)):
  model = build_compile_model(3, reg, learning_rate[lr])
  history = model.fit(tr_coords,tr_E,epochs=500,validation_data=(va_coords,va_E),verbose=0)
  va_predict=model.predict(va_coords)
  # get the RMSE of the model prediction for the validation set
  va_loss.append(np.sqrt(np.mean((va_predict-va_E)**2)))

#visualize the effects of different learning rate values on the model's accuracy
plt.plot(learning_rate, va_loss, marker='o')
plt.yscale("log")
plt.xscale("log")
plt.xlabel('Learning rate value')
plt.ylabel('Validation loss')
plt.show()

# get the learning rate value corresponding to the smallest validation error
for i in range(len(va_loss)):
    if va_loss[i]==min(va_loss):
        learn_rate=learning_rate[i]

## Learning curve

It would be interesting to show how the model would behave if trained on a smaller or larger data set. It is a way to tell if you need to gather more data to improve the model's accuracy or if it's saturated.

In [ ]:
training_points=[1500,1250,1000,750,500,250,100,50]
te_rmse=[]
for p in range(len(training_points)):
  ntrain=training_points[p]
  tr_coords=norm_coords[idx[:ntrain],:]
  tr_E=norm_E[idx[:ntrain]]

  model.fit(tr_coords,tr_E,epochs=1000,validation_data=(va_coords,va_E),verbose=0)

  norm_predict=model.predict(te_coords)
  predict=norm_predict*(np.max(ener) - np.min(ener))+np.mean(ener)
  te_rmse.append(np.sqrt(np.mean((predict-te_E)**2)))

plt.plot(training_points, te_rmse, marker='o')
plt.xlabel('Training points')
plt.ylabel('Model RMSE (Hartree)')
plt.show()